<a href="https://colab.research.google.com/github/TMhdSofyan/Convergence_Analysis/blob/main/Polynomial_Chaos_Expansion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/THESIS/Data/Experimental_Design2.csv')
df.tail()

,ID,Generator,Status,Result Status,Proxy Role,Keep SR3,Has SR3,Rating,CCPOR,KRG,...,PERMK_MULTI,POR_MULTI,SORW,Cum_Oil,Cum_Gas,Execution Node,Dataset Path,Optimal,MCMC Count,Comment
17072,17072,User,Complete,NormalTermination,Training,Auto,False,0,0.000005,0.531235,...,1.159760,0.997680,0.100961,1643914.0,317224130.0,77-1134-41,SA.cmsd\SA_17072.dat,False,0,NaN
17073,17073,User,Complete,NormalTermination,Training,Auto,False,0,0.000004,0.499895,...,0.870061,1.156218,0.108906,1661441.1,330066270.0,77-1134-41,SA.cmsd\SA_17073.dat,False,0,NaN
17074,17074,User,Complete,NormalTermination,Training,Auto,False,0,0.000004,0.509262,...,0.960209,0.982429,0.100172,1584137.9,315742180.0,77-1134-41,SA.cmsd\SA_17074.dat,False,0,NaN
17075,17075,User,Complete,NormalTermination,Training,Auto,False,0,0.000005,0.504134,...,1.041308,1.062378,0.090308,1632599.6,327702820.0,77-1134-41,SA.cmsd\SA_17075.dat,False,0,NaN
17076,17076,User,Complete,NormalTermination,Training,Auto,False,0,0.000005,0.431177,...,1.219194,1.158148,0.097388,1668201.6,328796160.0,77-1134-41,SA.cmsd\SA_17076.dat,False,0,NaN


In [6]:
df.columns

Index(['ID', 'Generator', 'Status', 'Result Status', 'Proxy Role', 'Keep SR3',
       'Has SR3', 'Rating', 'CCPOR', 'KRG', 'KRO', 'KRW', 'NG', 'NW',
       'PERMI_MULTI', 'PERMK_MULTI', 'POR_MULTI', 'SORW', 'Cum_Oil', 'Cum_Gas',
       'Execution Node', 'Dataset Path', 'Optimal', 'MCMC Count', 'Comment'],
      dtype='object')

In [7]:
!pip3 install UQpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.7/294.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 kB 9.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=a8b6defd6b2169cc9b131e0241725c456e81ddba42863ade116477959afa45b8
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [ ]:
df.columns

Index(['ID', 'Generator', 'Status', 'Result Status', 'Proxy Role', 'Keep SR3',
       'Has SR3', 'Rating', 'CCPOR', 'KRG', 'KRO', 'KRW', 'NG', 'NW',
       'PERMI_MULTI', 'PERMK_MULTI', 'POR_MULTI', 'SORW', 'Cum_Oil', 'Cum_Gas',
       'Execution Node', 'Dataset Path', 'Optimal', 'MCMC Count', 'Comment'],
      dtype='object')

In [8]:
df2 = df.drop(columns = ['ID', 'Generator', 'Status', 'Result Status', 'Proxy Role', 'Keep SR3',
       'Has SR3', 'Rating','KRG', 'KRO', 'KRW', 'NG', 'NW','PERMK_MULTI', 'SORW', 'Cum_Gas',
       'Execution Node', 'Dataset Path', 'Optimal', 'MCMC Count', 'Comment'])
df2.head()

,CCPOR,PERMI_MULTI,POR_MULTI,Cum_Oil
0,0.000005,1.001000,1.001000,1641502.0
1,0.000004,1.047771,1.100369,1658139.5
2,0.000004,1.021748,0.986743,1638181.8
3,0.000003,1.085006,0.985052,1637746.9
4,0.000005,0.893916,1.116380,1642039.0


In [44]:
first_1000 = df2.head(1000)
X = first_1000.drop(columns="Cum_Oil")
y = first_1000.Cum_Oil

In [45]:
X.head()

,CCPOR,PERMI_MULTI,POR_MULTI
0,0.000005,1.001000,1.001000
1,0.000004,1.047771,1.100369
2,0.000004,1.021748,0.986743
3,0.000003,1.085006,0.985052
4,0.000005,0.893916,1.116380


In [46]:
y.head()

0    1641502.0
1    1658139.5
2    1638181.8
3    1637746.9
4    1642039.0
Name: Cum_Oil, dtype: float64

In [47]:
from UQpy.distributions import Uniform, Normal, JointIndependent
from UQpy.surrogates import *

In [48]:
# Determine the number of samples from the DataFrame
n_samples = len(X)

# Define normal distributions for each parameter
ccpor_dist = Normal(loc=4.5e-6, scale=4.83579e-7)
permi_multi_dist = Normal(loc=1.001, scale=0.107569)
por_multi_dist = Normal(loc=1.001, scale=0.107569)

# Create a list of these normal distributions
marginals = [ccpor_dist, permi_multi_dist, por_multi_dist]

# Create a joint distribution object with independent marginals
joint_distribution = JointIndependent(marginals=marginals)

# Generate samples
samples = joint_distribution.rvs(n_samples)

# Convert the samples to a DataFrame to ensure the order matches
samples_df = pd.DataFrame(samples, columns=['CCPOR', 'PERMI_MULTI', 'POR_MULTI'])
samples_df.tail()

,CCPOR,PERMI_MULTI,POR_MULTI
995,0.000005,1.166274,0.864348
996,0.000004,1.033439,1.032226
997,0.000005,0.831124,1.084126
998,0.000004,1.081215,0.770844
999,0.000005,1.033330,0.858334


In [49]:
# Convert DataFrame to NumPy array for PCE input
x = samples_df.values
y = y.values  # Ensure y is a NumPy array

In [50]:
# Define the maximum degree for the polynomial basis
max_degree = 3

# Create polynomial basis using TotalDegreeBasis
polynomial_basis = TotalDegreeBasis(distributions=joint_distribution, max_degree=max_degree)

# Define the regression method
least_squares = LeastSquareRegression()

# Create the Polynomial Chaos Expansion (PCE) object
pce = PolynomialChaosExpansion(polynomial_basis=polynomial_basis, regression_method=least_squares)

# Fit the PCE model to the input (x) and output (y) data
pce.fit(x, y)

In [51]:
moments=pce.get_moments()
moments

(1634119.1511477674, 11429317.996269014)

In [52]:
# Optionally, print the PCE coefficients or any other relevant information
print("PCE Coefficients:")
print(pce.coefficients)

PCE Coefficients:
[[ 1.63411915e+06]
 [-3.77040714e+02]
 [ 9.40651822e+01]
 [-8.91006119e+02]
 [-3.83742650e+01]
 [ 1.12800929e+03]
 [-1.97023131e+02]
 [ 7.49392891e+02]
 [-4.36455927e+02]
 [-7.08460887e+02]
 [-8.70423752e+02]
 [ 7.94233944e+02]
 [-1.21382823e+03]
 [ 2.79969700e+01]
 [-1.87945899e+02]
 [ 6.91732426e+02]
 [ 1.29118234e+03]
 [-1.27697732e+01]
 [-1.15938281e+03]
 [-1.23685629e+03]]
